In [8]:
import json
import requests
import psycopg2
from time import gmtime, strftime

import pandas as pd
import pandas.io.sql as sqlio

### Query Bicing API + Insert into DB Locally

In [22]:
r = requests.get('http://wservice.viabicing.cat/v2/stations')
bicingJson = r.json()

In [26]:
#conexión a la base de datos
try:
    conn = psycopg2.connect("dbname='bicing' user='postgres' host='localhost' password='root'")
except:
    print("I am unable to connect to the database")

In [27]:
cursor = conn.cursor()
cursor.executemany("INSERT into bicing(id_station,lat,lon,slots,bikes) VALUES (%(id)s, %(latitude)s,%(longitude)s,%(slots)s,%(bikes)s)",bicingJson['stations'])
conn.commit()
cursor.close()

In [28]:
conn.close()

### External query of the DB

In [40]:
try:
    conn = psycopg2.connect("dbname='bicing' user='postgres' host='192.168.1.205' password='root'")
except:
    print("I am unable to connect to the database")
sql = "SELECT distinct(hora) FROM bicing ORDER BY hora ASC;"
sql = "SELECT * FROM bicing WHERE bikesdifference != 0 LIMIT 3000"
df = sqlio.read_sql_query(sql, conn)
conn.close()

In [41]:
df = df.assign(total_space = df.slots+df.bikes)
df.head()

,id,id_station,lat,lon,slots,bikes,hora,hmin,bikesdifference,total_space
0,371599,277,41.4083,2.16905,16,11,2018-10-23 12:09:03.052742,12:10,1,27
1,371627,305,41.3875,2.12364,18,9,2018-10-23 12:09:03.052742,12:10,-1,27
2,371638,316,41.4222,2.19832,25,2,2018-10-23 12:09:03.052742,12:10,-1,27
3,371661,339,41.4017,2.20528,18,7,2018-10-23 12:09:03.052742,12:10,1,25
4,371669,347,41.3595,2.14182,3,29,2018-10-23 12:09:03.052742,12:10,1,32


In [47]:
print(df.groupby('id_station')['total_space'].value_counts())

df['total_space'][df['id_station']==9]

id_station  total_space
1           29             12
2           22              4
            21              2
3           24             10
4           20             12
5           39              6
6           37              8
7           27              6
8           27             15
9           26              9
            27              7
11          21              8
            20              4
12          30             10
13          31              4
15          20             14
            19              1
16          21              7
17          21              5
18          27             14
19          28              6
            29              2
20          21              9
21          21              7
22          21              8
23          27             13
24          30             15
25          20              4
26          18              5
            17              4
                           ..
415         26              4
416         26  

625     27
678     27
734     27
835     27
1147    27
1516    26
1617    26
1720    26
1838    26
1898    26
2010    26
2076    26
2388    26
2448    26
2734    27
2955    27
Name: total_space, dtype: int64

In [51]:
df.iloc[[1147,1516]]

,id,id_station,lat,lon,slots,bikes,hora,hmin,bikesdifference,total_space
1147,381984,9,41.3845,2.18492,18,9,2018-10-23 12:32:02.923265,12:33,-1,27
1516,385225,9,41.3845,2.18492,18,8,2018-10-23 12:39:02.931840,12:40,-1,26
